<a href="https://colab.research.google.com/github/Ibtisam-a/Project-Implementation/blob/master/2_Bartels_Stewart_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bartels-Stewart Algorithm** 

# The algorithm is:

Compute the Schur forms $A = QUQ^*$ and $B=PSP*$.

Solve $U^*Y + YS = \hat{C}$ for $Y$, where $\hat{C} = P^*CQ$.

Compute $X=PYQ^*$.


# We can rewrite our equation TU + UT = F as


Compute the Schur form $T=PSP^*$.

Solve $S^*V + VS = \hat{F}$ for $V$, where $\hat{F} =  P^*FP$.

Compute $U=PVP^*$.

# **Instructions**

# There are two methods used:

**First,** Bartels-Stewart Algorithm which will be running by using cell 1,3 and 4.

**Second,** Bartels-Stewart Algorithm by useing  **scipy.linalg.solve_sylvester** , so you will run cell 1,2 and 4.

In [1]:
import numpy as np
from scipy.sparse import diags

#Our parameters
#number of internal nodes , unknowns, in each direction

N = 190  # our values: 190, 380, 760, 1520, 3040, 6080 and 12160

#number of internal nodes, unknowns, in each direction
n = N-2 
#step size 
h = 1/(n+1) 

#Matrix with zeros
U = np.zeros([n,n])

# x and y arrays between 0 and 1, spaced internal points
x = np.linspace(h, 1-h, n)
y = np.linspace(h, 1-h, n)

#Internal mesh without boundaries
X, Y = np.meshgrid(x, y, indexing='ij')

#our function 
F = np.sin(2*np.pi*X)*np.sin(2*np.pi*Y)  

#Tridiagonal matrix A
diagonals = [[1.5],[-0.5],[-0.5]]
A = np.multiply(1, diags(diagonals, [0, -1, 1], shape=(n, n)).toarray())

#Compute the exact solution on a specific time (0.00000000001) to be comapred 
Exact_sol = np.exp(-8*(np.pi*np.pi)*(0.00000000001))*np.sin(2*np.pi*X)*np.sin(2*np.pi*Y)




In [20]:
import numpy as np
from scipy.linalg import solve_sylvester
import time

def scipy_solve_sylvester(A, F):
    start_time = time.time()
    U = solve_sylvester(A, A, F)
    end_time = time.time()
    all_timess = end_time - start_time
    return U, all_timess


In [2]:
import numpy as np
import time
from scipy.linalg import schur




def bartels_stewart(A, F):
    n = len(A)
    start_time = time.time()

    #Compute Schur decomposition
    S, P = schur(A) 
    Stranspose = S.transpose()

    #Schur exexution time
    schur_time = time.time() - start_time
    
    #Solve S^*V + VS = F  for V, F = P^*FP
    V = np.empty([n,n])
    RHS = np.matmul(np.matmul(P.transpose(),F),P)
    for i in range(0,n):
        for j in range(0,n):                                
            V[i][j] = (RHS[i][j])/(Stranspose[i][i]+S[j][j])
    V_time = time.time() - schur_time - start_time

    #Compute U=PVP^*        
    U = np.matmul(np.matmul(P,V),P.transpose()) 
    #Calculate elapsed time
    end_time = time.time()
    solve_time = end_time - V_time - schur_time - start_time
    total_time = end_time - start_time
    all_timess = [schur_time, V_time, solve_time, total_time]
    return U,all_timess

In [ ]:
import numpy as np

def compute_error(A, F):   
    n = len(U)
    # The maximum difference between both solutions
    err_max_difference = 0  
    # The average difference between both solutions 
    err_ave_difference = 0

#For loop to compute the error
    for i in range(0,n):
        for j in range(0,n):
            err_ave_difference += np.absolute(Exact_sol[i][j] - U[i][j])**2
            if np.absolute(Exact_sol[i][j] - U[i][j]) > err_max_difference:
                err_max_difference = np.absolute(Exact_sol[i][j] - U[i][j])       
    err_ave_difference = (err_ave_difference * h**2)**0.5
    return err_max_difference, err_ave_difference

#Solving our system, and is based on which method we used.
#For example, if you use firts method you have to use (call) this function bartels_stewart(A, F), otherwise scipy_solve_sylvester(A, F)
U, all_timess = bartels_stewart(A, F)   #Based on which method you used  
err_max_difference, err_ave_difference = compute_error(U, Exact_sol)

print('Time taken:', all_timess)
print('Maximum diffence:', err_max_difference, '\n', 'Average difference:', err_ave_difference)